In [90]:
"""
DATE: 8/29/2020
AUTHOR: DANIEL WU
PURPOSE: Execute the webscraping algorithm for factiva reports 
LAST UPDATE:
"""

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Import Packages
import os
import time
import datetime
import pandas as pd
from getpass import getpass
import json

import import_ipynb
import importlib

def pause():
    programPause = input("Press the <ENTER> key to continue or type any letter to break loop")
    return programPause

In [91]:
# Webscraping Parameters
path_to_chromedriver = '/usr/local/bin/chromedriver'
usclib_url = 'https://libraries.usc.edu/'
usrnm = 'ddwu'
pw = getpass('USC Password: ')

download_folder = '/Users//user//Desktop//Jupyter Notebook//Factiva Scraping Test//WebScrapingAlgo//downloaded_articles'
articles_folder =  '/Users//user//Desktop//Jupyter Notebook//Factiva Scraping Test//WebScrapingAlgo//downloaded_articles'
working_dir = '/Users/user/Desktop/Jupyter Notebook/Factiva Scraping Test/WebScrapingAlgo'
os.chdir(working_dir)
factiva_query = '''(“equity” or “stock” or “share”) and (“offer” or “offering” or “issue” or “issuance” or “8K”)'''

USC Password: ········


In [98]:
# Read in Excel Spreadsheet
co_df = pd.read_excel('Bank_scraped.xlsx') 
co_df['conm'] = co_df['conm'].str.lower().str.split(',').str[0]
back_30 = datetime.timedelta(days=30)
next_3 = datetime.timedelta(days=3)

In [93]:
# Initialize Chrome Session
driver = webdriver.Chrome(executable_path = path_to_chromedriver)
driver.get(usclib_url)
driver.implicitly_wait(5) #seconds

In [94]:
# Import Procedural Modules
import procedural_commands as proc
importlib.reload(proc) 
import procedural_commands as proc

importing Jupyter notebook from procedural_commands.ipynb
importing Jupyter notebook from factiva_word_storage.ipynb


In [95]:
# Log onto the website
proc.log_onto_factiva(driver, usrnm, pw)

In [65]:
# Eventually create a triple nested dictionary for JSON File

# UNCOMMENT IF YOU REALLY WANT TO ERASE PROGRESS

# json_file = {}

In [99]:
# Pick up where last left off. Select last index here (from spreadsheet):
index_num = 1
start_index = index_num - 1    # Convert to Python Indexing
co_df = co_df.iloc[start_index:]

proc.send_query(driver, factiva_query)

# Enter keys and search algorithm:
for index, row in co_df.iterrows():
  
    cusip = str(row['cusip'])
        
    excel_index = index + 1    
        
    #re-scrape overwritten json files
    if row['Over_100'] != 1:
        continue
    if excel_index in (14, 68, 266, 270, 280, 318, 336, 381, 403, 413, 459, 531):
        continue
                        
    print(excel_index)
    json_file[excel_index] = {}      
    
    proc.enter_date(driver, start_index, index, row, next_3, back_30)        
    search_result = proc.enter_company_name(driver, row)
    
    if search_result == False:
        co_df['Scrape Comments'][index] = 'No search results for Company Name'
        co_df['Articles Scraped'][index] = 0
        continue
        
    proc.hit_search(driver)
        
    proc.get_articles(driver, working_dir, articles_folder, co_df, cusip, index, excel_index, json_file, 1)    
    
    try:
        driver.find_element_by_class_name('nextItem').click()
        time.sleep(2)
        proc.get_articles(driver, working_dir, articles_folder, co_df, cusip, index, excel_index, json_file, 2)    
        try:
            driver.find_element_by_class_name('nextItem').click()
            time.sleep(2)
            proc.get_articles(driver, working_dir, articles_folder, co_df, cusip, index, excel_index, json_file, 3)            
        except:
            pass
    except:
        pass
    
    time.sleep(2)
    
    try:
        proc.hit_next_search(driver) 
    except:        
        proc.hit_next_search(driver) 
        
    time.sleep(2)
    try:
        proc.clear_prev_search(driver)
    except:
        time.sleep(5)
        proc.clear_prev_search(driver)
    
#     x = pause()     
#     if x == '': 
#         continue
#     else: 
#         print("Loop Broken")
#         break
        

Manually clear query before proceeding.
Press the <ENTER> key to continue or type any letter to break loop
579
found wells fargo & company


procedural_commands.ipynb:283: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  "        prefixed = [filename for filename in os.listdir(working_dir) if filename.startswith(\"factiva_dl\")]                                        \n",


606
found toronto-dominion bank
622
found canadian imperial bank of commerce
688
found wells fargo & company


In [100]:
# JSON CODE HERE - DO THIS BEFORE LOGGING OFF JUPYTER NOTEBOOK

data_dump = open('json_file_banks_over100.txt', 'w', encoding = 'utf-8')
json.dump(json_file, data_dump, ensure_ascii = False)
data_dump.close()

# Write to excel progress made since last run (whether fail or success)

co_df.to_excel('Bank_rescraped.xlsx')

In [101]:
json_file.keys()

dict_keys([14, 68, 266, 270, 280, 318, 336, 381, 403, 413, 459, 531, 579, 606, 622, 688])